In [1]:
%load_ext autoreload
%autoreload 2
import uproot
import numpy as np
import awkward as ak
import matplotlib.pyplot as plt
import mplhep as hep
import hist

import sys
sys.path.append("../..")
from analyzer.dumperReader.reader import *

In [2]:
reader = DumperReader("/data_cms_upgrade/cuisset/supercls/alessandro_electrons/supercls-v1/dumper_101.root") #"../runs_results/histo.root:ticlDumper")

In [8]:
tracksters_zipped = reader.tracksters_zipped[["ts_id", "raw_pt", "raw_energy", "barycenter_eta"]]

In [19]:
sorted_ar = ak.sort(tracksters_zipped, ascending=False, axis=1,
        behavior={(np.less_equal, "trackster", "trackster",) : lambda x, y: x.raw_pt <= y.raw_pt})
sorted_ar

<Array [[{ts_id: 3377, ...}, ..., {...}], ...] type='100 * var * {ts_id: in...'>

In [17]:
sorted_ar[0]

<Array [{ts_id: 3377, raw_pt: 23, ...}, ...] type='3378 * {ts_id: int64, ra...'>

In [12]:
ak.singletons(ak.firsts(sorted_ar[sorted_ar.barycenter_eta <0]))

<Array [[{ts_id: 1657, raw_pt: ..., ...}], ...] type='100 * var * {ts_id: i...'>

In [ ]:
ak.concatenate([ak.singletons(ak.firsts(sorted_ar[sorted_ar.barycenter_eta <0])), ak.singletons(ak.firsts(sorted_ar[sorted_ar.barycenter_eta <0]))], axis=1)

<Array [[{raw_energy: 2.68, ...}, ...], ...] type='100 * var * {raw_energy:...'>

In [20]:
ts_ar = tracksters_getSeeds(reader.tracksters_zipped[["ts_id", "raw_pt", "raw_energy", "barycenter_eta"]])
ts_ar

<Array [[{ts_id: 558, ...}, {...}], ..., [...]] type='100 * var * {ts_id: i...'>

In [21]:
df = tracksters_toDf(ts_ar)
df

raw_pt  raw_energy  barycenter_eta
eventInternal ts_id                                       
0             558    23.049938  245.479675       -2.750928
              2242   16.149437  153.132401        2.673491
1             736    43.907665  138.009216       -1.773538
              2223   39.915539  155.163116        1.779360
2             757    19.182955   68.825623       -1.954545
...                        ...         ...             ...
97            2825   39.750000  226.456070        2.424081
98            749    62.088638  215.553238       -1.906013
              2349   54.445923  274.445282        1.895396
99            767    26.889877   96.964592       -1.901938
              2459   13.821290   79.518745        1.979664

[200 rows x 3 columns]

In [36]:
pd.options.display.min_rows = 20
reader.supercluster_df.reset_index(["supercls_id", "ts_in_supercls_id"]).set_index("ts_id", append=True).convert_dtypes()#.dtypes#.loc[0, 2938]

supercls_id  ts_in_supercls_id
eventInternal ts_id                                
0             558              0                  0
              89               0                  1
              2242             1                  0
              2172             1                  1
              211              2                  0
              1343             2                  1
              2308             3                  0
              2089             3                  1
              2592             3                  2
              2445             3                  3
...                          ...                ...
99            1802           266                  0
              1532           266                  1
              290            267                  0
              1090           267                  1
              2788           268                  0
              2156           268                  1
              2975           269                  0
              3106           269                  1
              3044           270                  0
              2170           270                  1

[56735 rows x 2 columns]

In [38]:
merged_df = pd.merge(df, reader.supercluster_df.reset_index(["supercls_id", "ts_in_supercls_id"]).set_index("ts_id", append=True).convert_dtypes(),
    left_index=True, right_index=True, #right_on=["eventInternal", "ts_id"], # #
    how="left")
merged_df

raw_pt  raw_energy  barycenter_eta  supercls_id  \
eventInternal ts_id                                                       
0             558    23.049938  245.479675       -2.750928            0   
              2242   16.149437  153.132401        2.673491            1   
1             736    43.907665  138.009216       -1.773538            0   
              2223   39.915539  155.163116        1.779360            1   
2             757    19.182955   68.825623       -1.954545            1   
              2711   29.842718  102.388222        1.926239            0   
3             846    37.598377  249.722977       -2.541424         <NA>   
              2302   34.947758  246.589294        2.366432            0   
4             562     5.966609   19.892654       -2.837314         <NA>   
              2539   22.151016  165.199677        2.486467            0   
...                        ...         ...             ...          ...   
95            889    11.048564   61.473553       -2.555406            0   
              2370    8.337068   41.546108        2.755267         <NA>   
96            1006    9.845423   34.299637       -1.799801            0   
              3092    8.087021   15.758677        2.356221         <NA>   
97            572    25.325029  119.897163       -2.431724            1   
              2825   39.750000  226.456070        2.424081            0   
98            749    62.088638  215.553238       -1.906013            0   
              2349   54.445923  274.445282        1.895396            1   
99            767    26.889877   96.964592       -1.901938            0   
              2459   13.821290   79.518745        1.979664            1   

                     ts_in_supercls_id  
eventInternal ts_id                     
0             558                    0  
              2242                   0  
1             736                    0  
              2223                   0  
2             757                    0  
              2711                   0  
3             846                 <NA>  
              2302                   0  
4             562                 <NA>  
              2539                   0  
...                                ...  
95            889                    0  
              2370                <NA>  
96            1006                   0  
              3092                <NA>  
97            572                    0  
              2825                   0  
98            749                    0  
              2349                   0  
99            767                    0  
              2459                   0  

[200 rows x 5 columns]

In [35]:
reader.supercluster_df

ts_id
eventInternal supercls_id ts_in_supercls_id       
0             0           0                    558
                          1                     89
              1           0                   2242
                          1                   2172
              2           0                    211
                          1                   1343
              3           0                   2308
                          1                   2089
                          2                   2592
                          3                   2445
...                                            ...
99            266         0                   1802
                          1                   1532
              267         0                    290
                          1                   1090
              268         0                   2788
                          1                   2156
              269         0                   2975
                          1                   3106
              270         0                   3044
                          1                   2170

[56735 rows x 1 columns]

In [34]:
merged_df.join(reader.supercluster_df, on=["eventInternal", "supercls_id"])

ValueError: len(left_on) must equal the number of levels in the index of "right"